In [1]:
import pandas as pd
import numpy as np

# ==========================================
# 1. CARGA E PREPARAÇÃO
# ==========================================
RAW_DATA_PATH = '../data/raw/ebt_municipios_br.xlsx'
print("🔄 Carregando dados...")

# Lê tudo como string para tratar vírgulas
df = pd.read_excel(RAW_DATA_PATH, dtype=str)
df.columns = [c.strip() for c in df.columns]

# Identificadores das colunas
col_municipio = next((c for c in df.columns if c.lower() in ['municipio', 'entidade avaliada', 'nome_municipio']), None)
col_uf = next((c for c in df.columns if c.lower() in ['uf', 'estado', 'sigla_uf']), None)
col_nota_final = next((c for c in df.columns if 'final' in c.lower() or 'avaliação' in c.lower()), None)

# Função de correção numérica
def fix_brazilian_number(val):
    if pd.isna(val): return 0.0
    val = str(val).replace(',', '.')
    try:
        return float(val)
    except:
        return 0.0

# Aplica correção numérica nas colunas de nota
for col in df.columns:
    if 'Nota' in col:
        df[col] = df[col].apply(fix_brazilian_number)

# ==========================================
# 2. SELEÇÃO DE DADOS
# ==========================================
df_rj = df[df[col_uf].str.strip() == 'Rio de Janeiro'].copy()
alvo = df_rj[df_rj[col_municipio].str.contains('Paracambi', na=False)].copy()

if alvo.empty: raise ValueError("Paracambi não encontrada.")

print(f"📍 Alvo: {alvo[col_municipio].values[0]} | Nota Geral: {alvo[col_nota_final].values[0]}")

# ==========================================
# 3. GERAÇÃO DE GAPS (REGRA EXPANDIDA)
# ==========================================
print("\n🕵️‍♂️ Rodando análise 'Pente Fino' (Qualquer ponto perdido conta)...")

comparativo_list = []

for i, col_nota in enumerate(df.columns):
    # Analisa apenas colunas de Nota (exceto a final)
    if 'Nota' in col_nota and col_nota != col_nota_final:
        
        # Dados da questão
        nota_paracambi = alvo[col_nota].values[0]
        max_nota_rj = df_rj[col_nota].max()
        
        # 🚨 NOVA LÓGICA:
        # Se a nota de Paracambi for MENOR que a máxima do estado, já é um GAP.
        # Antes só pegávamos se fosse igual a ZERO.
        gap = max_nota_rj - nota_paracambi
        
        # Só adiciona se houver gap E se a questão valer algum ponto (max > 0)
        if gap > 0 and max_nota_rj > 0:
            
            pergunta = df.columns[i-1] # Coluna anterior é o texto
            
            # Busca referência
            quem_gabaritou = df_rj[df_rj[col_nota] == max_nota_rj]
            exemplo = quem_gabaritou[~quem_gabaritou[col_municipio].str.contains('Governo|Capital', regex=True, na=False)]
            nome_exemplo = exemplo.iloc[0][col_municipio] if not exemplo.empty else quem_gabaritou.iloc[0][col_municipio]
            
            comparativo_list.append({
                'Questão Chave': pergunta,
                'Nota Paracambi': nota_paracambi,
                'Nota Máxima Possível': max_nota_rj,
                'Pontos Perdidos': gap,
                'Referência (Benchmarking)': nome_exemplo
            })

# ==========================================
# 4. SALVANDO RESULTADO
# ==========================================
df_gap = pd.DataFrame(comparativo_list)

if not df_gap.empty:
    # Ordena pelos que perderam mais pontos
    df_gap = df_gap.sort_values(by='Pontos Perdidos', ascending=False)
    
    print(f"\n📋 AGORA SIM: ENCONTRADOS {len(df_gap)} PONTOS DE MELHORIA.")
    print("Isso inclui notas zero e notas parciais (incompletas).")
    
    display(df_gap.head(10)) # Mostra prévia
    
    # Salva
    df_gap.to_excel('../data/processed/relatorio_final_gaps.xlsx', index=False)
    print("\n💾 Relatório salvo: data/processed/relatorio_final_gaps.xlsx")
    print("👉 Agora rode o Notebook 4.0 novamente para gerar o Laudo com essa lista maior!")
else:
    print("Paracambi é perfeita! (Nenhum gap encontrado).")

🔄 Carregando dados...
📍 Alvo: Paracambi - RJ | Nota Geral: 6.11

🕵️‍♂️ Rodando análise 'Pente Fino' (Qualquer ponto perdido conta)...

📋 AGORA SIM: ENCONTRADOS 9 PONTOS DE MELHORIA.
Isso inclui notas zero e notas parciais (incompletas).


,Questão Chave,Nota Paracambi,Nota Máxima Possível,Pontos Perdidos,Referência (Benchmarking)
7,24 - PEDIDO 2: O pedido 2 foi enviado com suce...,0.00,1.10,1.10,Niterói - RJ
8,25 - PEDIDO 3: O pedido 3 foi enviado com suce...,0.00,1.10,1.10,Campos dos Goytacazes - RJ
3,12 - CONTRATOS: O ente federado disponibiliza...,0.16,0.50,0.34,Angra dos Reis - RJ
2,11 - O ente federado disponibiliza consulta p...,0.00,0.30,0.30,Rio de Janeiro
5,14 - OBRAS PÚBLICAS: O ente federado disponibi...,0.00,0.25,0.25,Mesquita - RJ
4,13 - O ente federado permite gerar relatório d...,0.00,0.10,0.10,Rio de Janeiro
0,4 - UNIDADES ADMINISTRATIVAS: O ente federado ...,0.13,0.20,0.07,Rio de Janeiro
1,9 - LICITAÇÕES: O ente federado disponibiliza ...,0.23,0.30,0.07,Rio de Janeiro
6,19 - BASES DE DADOS ABERTOS: O ente federado ...,0.00,0.05,0.05,Macaé - RJ



💾 Relatório salvo: data/processed/relatorio_final_gaps.xlsx
👉 Agora rode o Notebook 4.0 novamente para gerar o Laudo com essa lista maior!


In [2]:
import pandas as pd

# ==========================================
# 1. CARGA DOS GAPS (GARANTINDO TUDO)
# ==========================================
DATA_PATH = '../data/processed/relatorio_final_gaps.xlsx'

try:
    df_gaps = pd.read_excel(DATA_PATH)
    total_erros = len(df_gaps)
    print(f"📂 Carregado relatório de Gaps. Total de problemas encontrados: {total_erros}")
except FileNotFoundError:
    print("❌ ERRO CRÍTICO: O arquivo 'relatorio_final_gaps.xlsx' não existe.")
    print("👉 Rode o notebook '3.0' novamente para gerar a lista de 30 itens antes de rodar este.")
    # Parar execução se não tiver arquivo
    raise

# ==========================================
# 2. O ESPECIALISTA JURÍDICO (REGRA DE NEGÓCIO)
# ==========================================
def consultar_especialista(pergunta):
    pergunta = str(pergunta).lower()
    
    # 1. e-SIC e Pedidos (O mais crítico)
    if any(x in pergunta for x in ['pedido', 'esic', 'eletrônico', 'físico', 'acompanhamento']):
        return {
            'Categoria': 'SISTEMA E-SIC (PASSIVA)',
            'Gravidade': '🔴 CRÍTICA',
            'Ação': 'O sistema de pedidos (e-SIC) não funciona ou falha na resposta. Instalar FalaBR ou corrigir fluxo.',
            'Lei': 'Lei 12.527 Art. 10'
        }
    
    # 2. Transparência Financeira
    elif any(x in pergunta for x in ['receitas', 'despesas', 'empenho', 'pagamento', 'diárias', 'obras']):
        return {
            'Categoria': 'FINANÇAS PÚBLICAS',
            'Gravidade': '🔴 CRÍTICA',
            'Ação': 'Integração em Tempo Real (máx 24h de atraso) das receitas e despesas. Publicar nominalmente.',
            'Lei': 'LC 131/2009 (Tempo Real)'
        }
    
    # 3. Compras e Licitações
    elif any(x in pergunta for x in ['licitação', 'contrato', 'edital', 'compra']):
        return {
            'Categoria': 'LICITAÇÕES E CONTRATOS',
            'Gravidade': '🟠 ALTA',
            'Ação': 'Disponibilizar íntegra de Editais e Contratos. Busca por ano, número e modalidade.',
            'Lei': 'Lei 12.527 Art. 8 §1º IV'
        }
    
    # 4. Servidores (RH)
    elif any(x in pergunta for x in ['servidores', 'folha', 'salários', 'remuneração']):
        return {
            'Categoria': 'RECURSOS HUMANOS',
            'Gravidade': '🟠 ALTA',
            'Ação': 'Publicar Folha de Pagamento nominal (Nome, Cargo, Salário Bruto/Líquido).',
            'Lei': 'Lei 12.527 Art. 8 §1º (Remuneração)'
        }
    
    # 5. Institucional
    elif any(x in pergunta for x in ['estrutura', 'endereço', 'telefone', 'horário', 'organograma']):
        return {
            'Categoria': 'INSTITUCIONAL',
            'Gravidade': '🟡 MÉDIA',
            'Ação': 'Atualizar Organograma, endereços e telefones das secretarias.',
            'Lei': 'Lei 12.527 Art. 8 §1º I'
        }

    # 6. Dados Abertos e Tecnologia
    elif any(x in pergunta for x in ['dados abertos', 'formato', 'csv', 'relatório estatístico']):
        return {
            'Categoria': 'TECNOLOGIA E FORMATOS',
            'Gravidade': '🟡 MÉDIA',
            'Ação': 'Disponibilizar dados em formatos não-proprietários (CSV, JSON, XML) e não apenas PDF.',
            'Lei': 'Lei 12.527 Art. 8 §3º'
        }
    
    else:
        return {
            'Categoria': 'GERAL / OUTROS',
            'Gravidade': '🟡 MÉDIA',
            'Ação': 'Verificar manualmente a disponibilidade no site (Regra dos 3 Cliques).',
            'Lei': 'Lei de Acesso à Informação'
        }

# ==========================================
# 3. PROCESSAMENTO EM MASSA
# ==========================================
print(f"🧠 A IA Jurídica está analisando todos os {total_erros} itens...")

analise_completa = []

for index, row in df_gaps.iterrows():
    pergunta = row['Questão Chave']
    # Tratamento de erro caso a coluna 'Referência' esteja vazia
    benchmark = row.get('Referência (Benchmarking)', 'Não informado') 
    
    parecer = consultar_especialista(pergunta)
    
    analise_completa.append({
        'ID': index + 1,
        'O Problema (Pergunta)': pergunta,
        'O Que Fazer (Solução)': parecer['Ação'],
        'Base Legal': parecer['Lei'],
        'Prioridade': parecer['Gravidade'],
        'Quem Copiar (Modelo)': benchmark
    })

df_final = pd.DataFrame(analise_completa)

# ==========================================
# 4. EXIBIÇÃO TOTAL (SEM FILTROS)
# ==========================================
# Ordenação Lógica: Crítico primeiro
ordem_map = {'🔴 CRÍTICA': 0, '🟠 ALTA': 1, '🟡 MÉDIA': 2}
df_final['rank'] = df_final['Prioridade'].map(ordem_map)
df_final = df_final.sort_values('rank').drop('rank', axis=1)

# Configurações para mostrar TUDO na tela
pd.set_option('display.max_rows', None)     # Mostra todas as linhas
pd.set_option('display.max_colwidth', None) # Não corta o texto
pd.set_option('display.width', 1000)

print(f"\n📋 RELATÓRIO COMPLETO GERADO ({len(df_final)} AÇÕES):")
display(df_final)

# Salvar
CAMINHO_FINAL = '../data/processed/LAUDO_COMPLETO_PARACAMBI.xlsx'
df_final.to_excel(CAMINHO_FINAL, index=False)
print(f"\n✅ Arquivo Excel Completo salvo em: {CAMINHO_FINAL}")

📂 Carregado relatório de Gaps. Total de problemas encontrados: 9
🧠 A IA Jurídica está analisando todos os 9 itens...

📋 RELATÓRIO COMPLETO GERADO (9 AÇÕES):


,ID,O Problema (Pergunta),O Que Fazer (Solução),Base Legal,Prioridade,Quem Copiar (Modelo)
0,1,24 - PEDIDO 2: O pedido 2 foi enviado com sucesso e obteve resposta?,O sistema de pedidos (e-SIC) não funciona ou falha na resposta. Instalar FalaBR ou corrigir fluxo.,Lei 12.527 Art. 10,🔴 CRÍTICA,Niterói - RJ
1,2,25 - PEDIDO 3: O pedido 3 foi enviado com sucesso e obteve resposta?,O sistema de pedidos (e-SIC) não funciona ou falha na resposta. Instalar FalaBR ou corrigir fluxo.,Lei 12.527 Art. 10,🔴 CRÍTICA,Campos dos Goytacazes - RJ
4,5,14 - OBRAS PÚBLICAS: O ente federado disponibiliza consulta para o acompanhamento de Obras Públicas?,O sistema de pedidos (e-SIC) não funciona ou falha na resposta. Instalar FalaBR ou corrigir fluxo.,Lei 12.527 Art. 10,🔴 CRÍTICA,Mesquita - RJ
2,3,12 - CONTRATOS: O ente federado disponibiliza possibilidade de consulta de informações sobre Contratos?,"Disponibilizar íntegra de Editais e Contratos. Busca por ano, número e modalidade.",Lei 12.527 Art. 8 §1º IV,🟠 ALTA,Angra dos Reis - RJ
5,6,13 - O ente federado permite gerar relatório da consulta de licitações ou da consulta de contratos em formato aberto?,"Disponibilizar íntegra de Editais e Contratos. Busca por ano, número e modalidade.",Lei 12.527 Art. 8 §1º IV,🟠 ALTA,Rio de Janeiro
3,4,11 - O ente federado disponibiliza consulta para acesso aos resultados das licitações ocorridas?,Verificar manualmente a disponibilidade no site (Regra dos 3 Cliques).,Lei de Acesso à Informação,🟡 MÉDIA,Rio de Janeiro
6,7,4 - UNIDADES ADMINISTRATIVAS: O ente federado disponibiliza informações sobre as suas unidades administrativas?,Verificar manualmente a disponibilidade no site (Regra dos 3 Cliques).,Lei de Acesso à Informação,🟡 MÉDIA,Rio de Janeiro
7,8,9 - LICITAÇÕES: O ente federado disponibiliza mecanismo ou ferramenta eletrônica de consulta de informações sobre Licitações?,Verificar manualmente a disponibilidade no site (Regra dos 3 Cliques).,Lei de Acesso à Informação,🟡 MÉDIA,Rio de Janeiro
8,9,19 - BASES DE DADOS ABERTOS: O ente federado publica em seu sítio oficial alguma relação das bases dedados abertos do município (catálago/inventário de dados abertos)?,"Disponibilizar dados em formatos não-proprietários (CSV, JSON, XML) e não apenas PDF.",Lei 12.527 Art. 8 §3º,🟡 MÉDIA,Macaé - RJ



✅ Arquivo Excel Completo salvo em: ../data/processed/LAUDO_COMPLETO_PARACAMBI.xlsx
